In [1]:
import ee
import geemap
import os
import numpy as np
import matplotlib.pyplot as plt
import geemap.colormaps as cm
# --
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [2]:
Map = geemap.Map()

gee_nzealand = ee.Geometry.Rectangle(173.65238652776756,-40.19969445726829,175.59620780852302,-38.385943001990896)
gee_zlimburg = ee.Geometry.Rectangle(5.221742442355838,50.321117197829594,6.67405336661114,51.67624746544328)
gee_gcanyon =ee.Geometry.Rectangle(-112.83420769522075,35.626994082857806,-111.5234288066205,36.85006285995321)
gee_tasmania = ee.Geometry.Rectangle(146.53872316173954,-43.26567341123374,147.92080639467764,-42.13416617520226)


# australiaDEM = ee.ImageCollection('AU/GA/AUSTRALIA_5M_DEM').select('elevation').filterBounds(gee_tasmania).mosaic()
image = ee.ImageCollection("LANDSAT/LC08/C02/T2_L2")
surface_temp = image.filterBounds(gee_gcanyon).select('ST_B10').filterDate('2018-01-01','2021-12-31')
temp_median = surface_temp.mean().multiply(0.00341802).add(149.0)

export_params = {
            "image": temp_median,
            "description": 'surface_temp_gc',
            "scale": 30,
            "region": gee_gcanyon,
            "folder": f"GEE_rf_export_surface_temp_GC",
            # 'file_per_band' : True
        }

print('Surface Temperature Collection (ST_B10):', surface_temp.getInfo())
Map.addLayer(temp_median, {}, 'RGB')
# 'bands': ['1'], 'min': 273, 'max': 373

geemap.ee_export_image_to_drive(**export_params)



Map.setCenter(146, -43, 7)
Map


Surface Temperature Collection (ST_B10): {'type': 'ImageCollection', 'bands': [], 'id': 'LANDSAT/LC08/C02/T2_L2', 'version': 1701299159480125, 'properties': {'type_name': 'ImageCollection', 'keywords': ['cfmask', 'cloud', 'fmask', 'global', 'l8sr', 'landsat', 'lasrc', 'lst', 'reflectance', 'sr', 'usgs'], 'visualization_1_bands': 'SR_B5,SR_B4,SR_B3', 'thumb': 'https://mw1.google.com/ges/dd/images/LANDSAT_SR_thumb.png', 'visualization_1_max': '30000.0', 'description': '<p>This dataset contains atmospherically corrected\nsurface reflectance and land surface temperature derived from the data\nproduced by the Landsat 8 OLI/TIRS sensors.\nThese images contain 5 visible and near-infrared (VNIR) bands and\n2 short-wave infrared (SWIR) bands processed to orthorectified surface\nreflectance, and one thermal infrared (TIR) band processed to orthorectified\nsurface temperature. They also contain intermediate bands used in\ncalculation of the ST products, as well as QA bands.</p><p>Landsat 8 SR pro

Map(center=[-43, 146], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=HBox(children=(T…

In [3]:
Map = geemap.Map()

# South Limburg
limburg = ee.Geometry.Rectangle(5.5986, 50.7287, 6.2014, 51.1737)
taranaki = ee.Geometry.Rectangle(173.834138, -39.468214, 174.316163, -39.147313)

def download_features(bbox, area):
    rf_features = {
            
        # "world_settlement_c": 
        #     ee.ImageCollection("projects/sat-io/open-datasets/WSF/WSF_2019").filterBounds(bbox).mosaic(),
        
        # "geomorhpon_c": 
        #     ee.ImageCollection("projects/sat-io/open-datasets/Geomorpho90m/geom").filterBounds(bbox).mosaic(),
        
        # "esri_lulc2020_c": 
        #     ee.ImageCollection(
        #     "projects/sat-io/open-datasets/landcover/ESRI_Global-LULC_10m").filterBounds(bbox).mosaic(),
        
        # "global_crops_c": 
        #     ee.ImageCollection("projects/sat-io/open-datasets/GFSAD/GCEP30").filterBounds(bbox).mosaic(),
        
        # "gl_surface_water_c": 
        #     ee.Image("projects/sat-io/open-datasets/ASTER/ASTWBD_ATT").clip(bbox),
            
        # "bare_earth_freq_n": 
        #     ee.Image("users/geocis/BareSurfaces/BF_1980_2019").clip(bbox),

        # "landsat_tree_caopy_cover_n":
        #     ee.ImageCollection("projects/sat-io/open-datasets/GFCC30TC").filterBounds(bbox).mosaic()
            
        # "global_water_c": 
        #     ee.Image("JRC/GSW1_4/GlobalSurfaceWater").clip(bbox).select('max_extent'),

        # "ndvi_n":
        #     ee.ImageCollection("MODIS/MYD09GA_006_NDVI").filter(ee.Filter.date("2022-01-01", "2023-01-01")).filterBounds(bbox).mosaic(),
        
        # "global_temp_au": 
        #     ee.ImageCollection("projects/sat-io/open-datasets/global-daily-air-temp/australia").filterBounds(bbox).mosaic(),
        
        # "global_temp_eu": 
        #     ee.ImageCollection("projects/sat-io/open-datasets/global-daily-air-temp/europe_asia").filterBounds(bbox).mosaic(),
        
        # "global_temp_us": 
        #     ee.ImageCollection("projects/sat-io/open-datasets/global-daily-air-temp/north_america").filterBounds(bbox).mosaic()
        "landsat_surface_temp":
            ee.ImageCollection("LANDSAT/LC08/C02/T2_L2")
            .filterBounds(bbox)
            .filterDate('2023-05-01', '2023-05-10')
            .select('SR_QA_AEROSOL')
            .mosaic()
    }

    for feature in rf_features:
        export_params = {
            "image": rf_features[feature],
            "description": f"{feature}",
            "scale": 30,
            "region": bbox,
            "folder": f"GEE_rf_export_{area}",
            # 'file_per_band' : True
        }
        geemap.ee_export_image_to_drive(**export_params)

# Download Features from locations below:
locations = {
    'greater_nzealand': ee.Geometry.Rectangle(173.062973, -39.974479, 175.172348, -38.734258),
    'greater_zlimburg': ee.Geometry.Rectangle(4.855957, 50.534380, 6.624756, 51.412912),
    'greater_gcanyon': ee.Geometry.Rectangle(-112.276840,35.977451,-111.787262,36.291884)
}

gee_nzealand = ee.Geometry.Rectangle(173.65238652776756,-40.19969445726829,175.59620780852302,-38.385943001990896)
gee_zlimburg = ee.Geometry.Rectangle(5.221742442355838,50.321117197829594,6.67405336661114,51.67624746544328)
gee_gcanyon =ee.Geometry.Rectangle(-112.83420769522075,35.626994082857806,-111.5234288066205,36.85006285995321)
gee_tasmania = ee.Geometry.Rectangle(146.53872316173954,-43.26567341123374,147.92080639467764,-42.13416617520226)

# download_features(gee_gcanyon, 'gee_gcanyon')
# download_features(gee_zlimburg, 'gee_zlimburg')
# download_features(gee_nzealand, 'gee_nzealand')
download_features(gee_tasmania, 'tasmania')


In [ ]:
Map.addLayer()